<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [102]:
import pandas as pd
import psycopg2

# Для подавления сообщений об ошибках при обработке pandas ответов psycopg2
import warnings
warnings.filterwarnings('ignore')

In [103]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [104]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [23]:
# текст запроса
query_3_1 = f'''
             SELECT COUNT(*)
             FROM public.vacancies
'''

In [25]:
# результат запроса
df_3_1 = pd.read_sql_query(query_3_1, connection)
df_3_1

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [29]:
# текст запроса
query_3_2 = f'''
             SELECT COUNT(*)
             FROM public.employers e
'''

In [30]:
# результат запроса
df_3_2 = pd.read_sql_query(query_3_2, connection)
df_3_2

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [32]:
# текст запроса
query_3_3 = f'''
             SELECT COUNT(*)
             FROM public.areas a
'''

In [33]:
# результат запроса
df_3_3 = pd.read_sql_query(query_3_3, connection)
df_3_3

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [34]:
# текст запроса
query_3_4 = f'''
             SELECT COUNT(*)
             FROM public.industries i
'''

In [35]:
# результат запроса
df_3_4 = pd.read_sql_query(query_3_4, connection)
df_3_4

,count
0,294


***

### Выводы по предварительному анализу данных:
В результате нашего предварительного анализа мы видим, что количество вакансий примерно в 2 раза превышает количество работодателей. В базе представлено 1362 региона - охвачена довольно широкая география поиска работы. Представлено 294 различные сферы деятельности. Это неплохая статистика показывает, что у соискателей есть возможность выбора и больше шансов устроиться.
Будем проводить дополнительные исследования ниже.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [62]:
# текст запроса
query_4_1 = f'''
             SELECT 
                 a.name as area,
                 count(v.id) as vacancies_count
             FROM 
                 public.areas as a
                 JOIN public.vacancies as v on v.area_id = a.id
             GROUP BY a.name
             ORDER BY count(v.id) desc
             LIMIT 15
'''

In [63]:
# результат запроса
df_4_1 = pd.read_sql_query(query_4_1, connection)
df_4_1

,area,vacancies_count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [64]:
# текст запроса
query_4_2 = f'''
             SELECT 
                 count(*)
             FROM 
                 public.vacancies
                 WHERE salary_from is not null
                 OR salary_to is not null
'''

In [65]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [66]:
# текст запроса
query_4_3 = f'''
             SELECT
                 round(avg(v.salary_from)) avg_salary_from, 
                 round(avg(v.salary_to)) avg_salary_to
             FROM public.vacancies v
'''


In [67]:
# результат запроса
df_4_3 = pd.read_sql_query(query_4_3, connection)
df_4_3

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [68]:
# текст запроса
query_4_4 = f'''
             SELECT
                 count(*),
                 v.schedule,
                 v.employment
             FROM public.vacancies v
             GROUP BY 2,3
             ORDER BY 1 desc
'''

In [69]:
# результат запроса
df_4_4 = pd.read_sql_query(query_4_4, connection)
df_4_4

,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [70]:
# текст запроса
query_4_5 = f'''
             SELECT
                 count(*),
                 v.experience
             FROM public.vacancies v
             GROUP BY 2
             ORDER BY 1 
'''

In [71]:
# результат запроса
df_4_5 = pd.read_sql_query(query_4_5, connection)
df_4_5

,count,experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


4*. Определим, в какой отрасли можно найти больше всего вакансий

In [72]:
ad_query = f'''
             SELECT 
                 i.name as industry,
                 count(v.id) as vacancies_count
             FROM
                public.employers_industries as ei
                join public.industries as i on i.id = ei.industry_id
                join public.vacancies as v on v.employer_id = ei.employer_id
             GROUP BY i.id
             ORDER BY count(v.id) desc
             LIMIT 5
                
'''

In [73]:
df = pd.read_sql_query(ad_query, connection)
df

,industry,vacancies_count
0,Разработка программного обеспечения,12499
1,"Системная интеграция, автоматизации технологи...",11034
2,"Интернет-компания (поисковики, платежные систе...",6413
3,Банк,2742
4,Мобильная связь,1585


***

### Выводы по детальному анализу вакансий:
После детального анализа вакансий мы видим, что основная доля вакансий сосредоточена в крупных городах России и стран СНГ: в топ-5 по числу вакансий входят Москва, Санкт-Петербург, Минск, Новосибирск и Алматы (на данные города приходится примерно 30% всех вакансий). Около 70% вакансий имеют тип рабочего графика "полный день" и тип трудоустройства "полная занятость". Наиболее частые пожелания работодателей по опыту работы (50% вакансий) - от 1 до 3 лет. Для ряда должностей, связанных с занимаением каких-либо руководящих позиций, может потребоваться уже больше опыта - от 3 до 6 лет (около 30% вакансий). Ещё можно заметить, что нижняя граница зарплатной вилки имеет довольно высокое среднее значение (для средней полосы России и стран СНГ) - более 70 тыс. рублей в месяц. Вероятно, это также связано с тем, что основная доля вакансий сосредоточена в крупных городах и столицах. Основное количество вакансий приходится на IT-технологии (разработка ПО, системная интеграция и автоматизация, Интернет-компания, мобильная связь) и банковскую сферу. На эти топ-5 отраслей приходится около 70% от общего числа вакансий.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [79]:
# текст запроса
query_5_1 = f'''
             SELECT
                 count(*),
                 e.name
             FROM public.vacancies v
                 JOIN public.employers e on e.id = v.employer_id
             GROUP BY 2
             ORDER BY 1 desc
             LIMIT 10
'''

In [80]:
# результат запроса
df_5_1 = pd.read_sql_query(query_5_1, connection)
df_5_1

,count,name
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть
5,327,ИК СИБИНТЕК
6,292,МТС
7,247,DataArt
8,204,Совкомбанк Технологии
9,176,Первый Бит


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [85]:
# текст запроса
query_5_2 = f'''
             SELECT
                 a.name,
                 count(e.id),
                 count(v.id)
             FROM public.areas a
                 LEFT JOIN public.vacancies v on a.id = v.area_id
                 LEFT JOIN public.employers e on a.id = e.area
                 WHERE v.id is null
             GROUP BY 1
             ORDER BY 2 desc
             LIMIT 10
'''

In [86]:
# результат запроса
df_5_2 = pd.read_sql_query(query_5_2, connection)
df_5_2

,name,count,count
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
5,Ростовская область,18,0
6,Азербайджан,17,0
7,Нижегородская область,16,0
8,Республика Татарстан,16,0
9,Узбекистан,15,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [87]:
# текст запроса
query_5_3 = f'''
             SELECT
                 e.name,
                 count(distinct v.area_id)
             FROM public.employers e  
                 JOIN public.vacancies v on v.employer_id = e.id
             GROUP BY 1
             ORDER BY 2 desc
             LIMIT 10
'''

In [88]:
# результат запроса
df_5_3 = pd.read_sql_query(query_5_3, connection)
df_5_3

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [89]:
# текст запроса
query_5_4 = f'''
              SELECT
                  count(e.name)
             FROM public.employers e  
                 LEFT JOIN public.employers_industries ei on ei.employer_id = e.id
                 WHERE ei.industry_id is null
'''

In [90]:
# результат запроса
df_5_4 = pd.read_sql_query(query_5_4, connection)
df_5_4

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [91]:
# текст запроса
query_5_5 = f'''
             SELECT
                 e.name,
                 count(ei.industry_id)
             FROM public.employers e  
                 LEFT JOIN public.employers_industries ei on ei.employer_id = e.id
             GROUP BY 1
             HAVING count(ei.industry_id) = 4
             ORDER BY 1
             LIMIT 5
'''

In [92]:
# результат запроса
df_5_5 = pd.read_sql_query(query_5_5, connection)
df_5_5

,name,count
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [93]:
# текст запроса
query_5_6 = f'''
             SELECT
                 count(e.id)
             FROM public.employers e  
                 LEFT JOIN public.employers_industries ei on ei.employer_id = e.id
                 LEFT JOIN public.industries i on ei.industry_id = i.id
                 WHERE i.name = 'Разработка программного обеспечения'
'''

In [94]:
# результат запроса
df_5_6 = pd.read_sql_query(query_5_6, connection)
df_5_6

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [99]:
# код для получения списка городов-милионников
# REQUIRED pip install requests
import requests
# REQUIRED pip install beautifulsoup4
from bs4 import BeautifulSoup as bs

wiki_url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'

response = requests.get(wiki_url)
page = bs(response.text, 'html.parser')
megas_list = []
tag_list = page.find('table', class_='standard sortable').find_all('a')
for tag_ in tag_list:
    str_ = tag_.text
    if str_[0].isupper():
        megas_list.append(str_)
megas_str = "('" + "', '".join(megas_list) + "')"
print(megas_str)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [107]:
# текст запроса
query_5_7 = f'''
             SELECT
                 a.name cities,
                 count(v.id)
             FROM public.vacancies v
                 LEFT JOIN public.employers e on v.employer_id = e.id
                 LEFT JOIN public.areas a on a.id = v.area_id
                 WHERE e.name = 'Яндекс' and a.name in {m_city}
             GROUP BY a.name
             UNION ALL
             SELECT
                 'Total', count(distinct v.id)
             FROM public.vacancies v
                 LEFT JOIN public.employers e on v.employer_id = e.id
                 LEFT JOIN public.areas a on a.id = v.area_id
                 WHERE e.name = 'Яндекс' and a.name in {m_city}
             ORDER BY 2           
'''

In [108]:
# результат запроса
df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7

,cities,count
0,Омск,21
1,Красноярск,23
2,Пермь,25
3,Казань,25
4,Самара,26
5,Новосибирск,35
6,Нижний Новгород,36
7,Екатеринбург,39
8,Санкт-Петербург,42
9,Москва,54


***

In [ ]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса

In [ ]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса

In [ ]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса

In [ ]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса

In [ ]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

***

In [ ]:
# выводы по предметному анализу

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования